# 🚀 SLM Fine-Tuning Part 3: Training Dynamics

**What you'll learn:**
- What loss curves tell you about training
- How learning rate schedules affect convergence
- How to use Weights & Biases for visualization
- When training is working vs failing

**Setup:**
- Model: `Qwen/Qwen2.5-0.5B-Instruct`
- Dataset: `mlabonne/guanaco-llama2-1k` (1,000 samples)
- GPU: Colab T4 (15GB VRAM)
- Time per run: ~10-15 minutes

---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1️⃣ Setup & Installation

First, let's install the required libraries.

In [ ]:
!pip install -q transformers datasets peft trl accelerate bitsandbytes wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.0 MB/s eta 0:00:00


## 2️⃣ Weights & Biases Setup

W&B will create beautiful training charts automatically.

**First time?**
1. Go to [wandb.ai](https://wandb.ai) and create a free account
2. Run the cell below
3. Paste your API key when prompted

In [ ]:
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 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


wandb: WARNING Invalid choice
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: balci-pelin (balci-pelin-patolojiai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## 3️⃣ Check GPU

Make sure you're connected to a GPU runtime.

**No GPU?** Go to `Runtime` → `Change runtime type` → Select `T4 GPU`

In [ ]:
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU: {gpu_name}")
    print(f"✅ VRAM: {gpu_memory:.1f} GB")
else:
    print("❌ No GPU detected! Change runtime to GPU.")

✅ GPU: Tesla T4
✅ VRAM: 15.8 GB


## 4️⃣ Configuration

All settings in one place. We'll modify `LR_SCHEDULE` for each experiment.

In [ ]:
# ============================================================
# CONFIGURATION
# ============================================================

# Model & Data
MODEL_ID = "Qwen/Qwen2.5-0.5B-Instruct"
DATASET_ID = "mlabonne/guanaco-llama2-1k"

# Training Settings
MAX_SEQ_LEN = 512
BATCH_SIZE = 2
GRAD_ACCUM = 8          # Effective batch = 2 * 8 = 16
MAX_STEPS = 200         # Enough to see learning dynamics
LEARNING_RATE = 2e-4

# Learning Rate Schedule (we'll change this for each run)
# Options: "constant", "cosine", "linear"
LR_SCHEDULE = "constant"
WARMUP_RATIO = 0.0      # We'll set to 0.1 for warmup experiments

# LoRA Settings (keep fixed for this experiment)
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

# Logging
LOGGING_STEPS = 5       # Log every 5 steps for smooth curves

# Output
RUN_NAME = "constant_lr"  # Change for each run
OUTPUT_DIR = f"./outputs/{RUN_NAME}"

print("✅ Configuration loaded")
print(f"   Model: {MODEL_ID}")
print(f"   Dataset: {DATASET_ID}")
print(f"   LR Schedule: {LR_SCHEDULE}")
print(f"   Warmup: {WARMUP_RATIO*100}%")

✅ Configuration loaded
   Model: Qwen/Qwen2.5-0.5B-Instruct
   Dataset: mlabonne/guanaco-llama2-1k
   LR Schedule: constant
   Warmup: 0.0%


## 5️⃣ Load Dataset

The Guanaco dataset contains instruction-following conversations.

Let's look at what the data looks like.

In [ ]:
from datasets import load_dataset

dataset = load_dataset(DATASET_ID, split="train")

print(f"✅ Dataset loaded: {len(dataset)} samples")
print(f"\n📝 Example:")
print("-" * 50)
print(dataset[0]["text"][:500] + "...")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-9ad84bb9cf65a4(…):   0%|          | 0.00/967k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

✅ Dataset loaded: 1000 samples

📝 Example:
--------------------------------------------------
<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal...


## 6️⃣ Load Model & Tokenizer

We load in float16 to save VRAM.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto",
)

print(f"✅ Model loaded: {MODEL_ID}")
print(f"   Parameters: {model.num_parameters():,}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

✅ Model loaded: Qwen/Qwen2.5-0.5B-Instruct
   Parameters: 494,032,768


## 7️⃣ Apply LoRA

Freeze the base model and add trainable adapter layers.

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,081,344 || all params: 495,114,112 || trainable%: 0.2184


## 8️⃣ Training Arguments

This is where we configure the learning rate schedule.

### Understanding LR Schedules

| Schedule | Behavior | When to use |
|----------|----------|-------------|
| `constant` | LR stays the same | Quick experiments |
| `linear` | LR decreases linearly to 0 | Conservative training |
| `cosine` | LR follows cosine curve to ~0 | Most common choice |

**Warmup** gradually increases LR at the start to stabilize training.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,

    # Batch settings
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,

    # Training length
    max_steps=MAX_STEPS,

    # Learning rate settings ⭐
    learning_rate=LEARNING_RATE,
    lr_scheduler_type=LR_SCHEDULE,    # "constant", "cosine", "linear"
    warmup_ratio=WARMUP_RATIO,        # 0.0 = no warmup, 0.1 = 10% warmup

    # Precision
    fp16=True,

    # Logging ⭐
    logging_steps=LOGGING_STEPS,
    report_to="wandb",                # Send logs to W&B
    run_name=RUN_NAME,                # Name in W&B dashboard

    # Checkpoints (we'll cover this in Part 6)
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,

    # Misc
    optim="adamw_torch",
    seed=42,                          # Reproducibility
)

print("✅ Training arguments configured")
print(f"   LR Schedule: {LR_SCHEDULE}")
print(f"   Warmup: {WARMUP_RATIO*100}%")
print(f"   Effective batch size: {BATCH_SIZE * GRAD_ACCUM}")

✅ Training arguments configured
   LR Schedule: constant
   Warmup: 0.0%
   Effective batch size: 16


## 9️⃣ Create Trainer

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer,
)

print("✅ Trainer ready")

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:2111: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


✅ Trainer ready


## 🔟 Train!

This will take ~10-15 minutes.

**While training:**
- Open your [W&B dashboard](https://wandb.ai) in another tab
- Watch the loss curve update in real-time

In [ ]:
print(f"🚀 Starting training: {RUN_NAME}")
print(f"   Watch live at: https://wandb.ai")
print("-" * 50)

trainer.train()

print("-" * 50)
print(f"✅ Training complete: {RUN_NAME}")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


🚀 Starting training: constant_lr
   Watch live at: https://wandb.ai
--------------------------------------------------


Step,Training Loss
5,1.791500
10,1.802100
15,1.796000
20,1.781400
25,1.783000
30,1.708300
35,1.721500
40,1.622300
45,1.702000
50,1.759500


--------------------------------------------------
✅ Training complete: constant_lr


## 1️⃣1️⃣ Save the Adapter

In [ ]:
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Adapter saved to: {OUTPUT_DIR}")

✅ Adapter saved to: ./outputs/constant_lr


## 1️⃣2️⃣ Finish W&B Run

In [ ]:
wandb.finish()
print("✅ W&B run finished")

train/entropy,▄▇▇█▆▄▅▂▆▆▄▆▄▂▆▁▂▃▅▃█▃▄▂▃▁▁▄▄▂▅▅▄▅▁▂▂▅▃▃
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,██▅▄▄▆▂▃▅▃▂▂▁▃▄▂▁▂▃▃▃▃▂▂▃▂▁▄▃▃▄▄▆▆▅▂▄▂▄▅
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,███▇▇▅▆▃▅▇▅▆▄▂▆▄▄▅▅▄█▄▅▂▃▁▂▄▅▁▅▄▅▆▂▂▃▅▂▅
train/mean_token_accuracy,▃▃▁▁▃▅▃▅▃▃▄▃▅▇▃▇▆▅▅▆▁▅▅▆▆▇█▅▄▇▄▄▅▄█▇▇▃▇▆
train/num_tokens,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
total_flos,3430423476632064.0
train/entropy,1.72361
train/epoch,3.176


✅ W&B run finished


---

# 🔬 Experiment Time!

Now let's run the same training with different LR schedules and compare.

## How to Run Multiple Experiments

**Option A: Restart and modify**
1. Go to `Runtime` → `Restart runtime`
2. Change `LR_SCHEDULE` and `RUN_NAME` in the config cell
3. Run all cells again

**Option B: Use the quick-run cell below**

The cell below lets you run a new experiment without restarting.

In [ ]:
# ============================================================
# QUICK RUN: Change settings and run this cell
# ============================================================

def run_experiment(lr_schedule, warmup_ratio, run_name):
    """Run a training experiment with specified LR schedule."""
    import gc

    # Clear memory from previous run
    gc.collect()
    torch.cuda.empty_cache()

    # Reload model fresh
    print(f"\n{'='*60}")
    print(f"🚀 Starting experiment: {run_name}")
    print(f"   LR Schedule: {lr_schedule}")
    print(f"   Warmup: {warmup_ratio*100}%")
    print(f"{'='*60}\n")

    # Load fresh model
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    model = get_peft_model(model, lora_config)

    # Training args with new schedule
    args = TrainingArguments(
        output_dir=f"./outputs/{run_name}",
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACCUM,
        max_steps=MAX_STEPS,
        learning_rate=LEARNING_RATE,
        lr_scheduler_type=lr_schedule,
        warmup_ratio=warmup_ratio,
        fp16=True,
        logging_steps=LOGGING_STEPS,
        report_to="wandb",
        run_name=run_name,
        save_strategy="steps",
        save_steps=100,
        save_total_limit=2,
        optim="adamw_torch",
        seed=42,
    )

    # Create trainer
    trainer = SFTTrainer(
        model=model,
        args=args,
        train_dataset=dataset,
        processing_class=tokenizer,
    )

    # Train
    trainer.train()

    # Save
    trainer.model.save_pretrained(f"./outputs/{run_name}")

    # Finish W&B
    wandb.finish()

    print(f"\n✅ Experiment complete: {run_name}")

    # Cleanup
    del model, trainer
    gc.collect()
    torch.cuda.empty_cache()

print("✅ run_experiment() function ready")
print("\nRun experiments with:")
print('  run_experiment("constant", 0.0, "exp1_constant")')
print('  run_experiment("cosine", 0.0, "exp2_cosine")')
print('  run_experiment("cosine", 0.1, "exp3_cosine_warmup")')

✅ run_experiment() function ready

Run experiments with:
  run_experiment("constant", 0.0, "exp1_constant")
  run_experiment("cosine", 0.0, "exp2_cosine")
  run_experiment("cosine", 0.1, "exp3_cosine_warmup")


## Run Experiment 1: Constant LR

In [ ]:
run_experiment("constant", 0.0, "exp1_constant")


🚀 Starting experiment: exp1_constant
   LR Schedule: constant
   Warmup: 0.0%



/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:2111: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
5,1.791500
10,1.801200
15,1.795600
20,1.782300
25,1.783100
30,1.707400
35,1.721900
40,1.622500
45,1.702300
50,1.760300


train/entropy,▅▇▇▇▆▄▅▂▆▆▄▆▄▂▆▂▂▃▅▃█▃▄▂▃▁▁▄▄▂▅▅▄▅▁▂▃▅▃▃
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,██▅▅▄▆▂▃▅▃▃▂▁▃▄▂▂▃▃▄▄▃▃▃▄▂▁▄▄▃▄▄▆▇▆▃▄▂▄▆
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,███▇▇▅▆▃▅▇▅▆▄▂▆▄▄▅▅▄█▄▅▂▃▁▂▄▅▁▅▄▅▆▂▂▃▅▂▅
train/mean_token_accuracy,▃▃▁▁▃▅▃▆▂▃▄▃▅▇▃▇▆▅▅▆▁▅▅▆▆▇▇▅▄▇▄▄▅▄█▇▇▃▇▆
train/num_tokens,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
total_flos,3430423476632064.0
train/entropy,1.72372
train/epoch,3.176



✅ Experiment complete: exp1_constant


## Run Experiment 2: Cosine Decay (no warmup)

In [ ]:
run_experiment("cosine", 0.0, "exp2_cosine")


🚀 Starting experiment: exp2_cosine
   LR Schedule: cosine
   Warmup: 0.0%



/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:2111: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
5,1.791500
10,1.801300
15,1.795900
20,1.782600
25,1.783400
30,1.707500
35,1.722100
40,1.622700
45,1.702800
50,1.760500


train/entropy,▄▇▇▇▆▄▅▂▆▆▃▆▄▂▆▁▂▃▅▃█▃▅▂▂▁▁▄▄▂▅▅▄▅▁▂▃▅▄▄
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,██▅▅▄▆▂▃▅▃▃▂▁▃▄▂▂▂▄▄▄▃▂▃▄▂▁▃▃▃▃▃▅▅▄▂▃▁▃▄
train/learning_rate,███████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
train/loss,███▇▇▅▆▃▅▇▄▅▃▂▆▃▄▄▄▄█▄▅▂▃▁▂▄▆▁▅▄▅▆▂▃▃▆▃▆
train/mean_token_accuracy,▃▃▂▂▃▆▃▆▃▃▄▃▅▇▃▇▆▅▅▆▁▅▅▆▆█▇▅▄▇▄▄▅▄█▇▇▃▆▅
train/num_tokens,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
total_flos,3430423476632064.0
train/entropy,1.79142
train/epoch,3.176



✅ Experiment complete: exp2_cosine


## Run Experiment 3: Cosine Decay + 10% Warmup

In [ ]:
run_experiment("cosine", 0.1, "exp3_cosine_warmup")


🚀 Starting experiment: exp3_cosine_warmup
   LR Schedule: cosine
   Warmup: 10.0%



/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:2111: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
5,1.825700
10,1.879800
15,1.892300
20,1.857400
25,1.832800
30,1.731800
35,1.733900
40,1.631600
45,1.709800
50,1.764100


train/entropy,▄▅▆█▇▅▅▂▆▅▃▄▄▂▅▁▂▃▄▂▇▃▄▂▂▁▁▃▄▂▄▄▄▅▁▂▃▅▄▄
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,█▅▃▂▂▃▂▂▃▂▂▁▁▂▂▁▁▁▂▂▂▂▁▂▂▁▁▂▂▁▂▂▃▂▂▁▂▁▂▂
train/learning_rate,▂▄▆███████▇▇▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
train/loss,▇██▇▇▅▅▂▄▅▄▄▃▂▅▃▃▄▄▃▆▃▄▂▃▁▂▃▄▁▄▃▄▅▂▂▂▄▃▅
train/mean_token_accuracy,▄▃▁▁▃▅▄▆▃▄▅▄▆▇▃▇▆▅▅▇▂▅▆▇▆██▅▅▇▄▅▆▄█▇▇▄▆▅
train/num_tokens,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
total_flos,3430423476632064.0
train/entropy,1.79131
train/epoch,3.176



✅ Experiment complete: exp3_cosine_warmup


---

# 📊 Analyzing Results

## In W&B Dashboard

1. Go to [wandb.ai](https://wandb.ai) → Your project
2. Click on a run to see its metrics
3. To compare runs:
   - Select multiple runs (checkboxes)
   - Click "Add to comparison"
   - View overlaid loss curves

## What to Look For

### Loss Curve Shapes

```
Good training:          Overfitting:            Unstable:
                        
Loss                    Loss                    Loss
  │╲                      │╲                      │╲  /╲
  │ ╲                     │ ╲___                  │ ╲/  ╲/╲
  │  ╲___                 │     ╲__/──            │       ╲/
  │      ╲___             │                       │
  └──────────             └──────────             └──────────
  Smooth decrease         Goes up = overfit       Spiky = LR too high
```

### Comparing LR Schedules

| Schedule | Expected behavior |
|----------|------------------|
| Constant | Steady decrease, might plateau |
| Cosine | Faster early, slows down nicely |
| Cosine + Warmup | Slower start, then smooth curve |

---

# 🧠 Key Takeaways

## 1. Loss Tells You If Training Works
- **Decreasing loss** = model is learning
- **Flat loss** = learning rate too low, or model saturated
- **Increasing loss** = learning rate too high, or overfitting
- **Spiky loss** = batch too small, or LR too high

## 2. LR Schedule Matters
- **Constant**: Simple, but may not converge well
- **Cosine**: Usually best default choice
- **Warmup**: Helps with large models or unstable starts

## 3. Effective Batch Size = Stability
- Larger effective batch → smoother gradients → more stable training
- We used: `batch_size=2 × grad_accum=8 = 16`

## 4. Logging Frequency Matters
- Too infrequent → miss important dynamics
- Too frequent → noisy curves
- `logging_steps=5` is a good balance

---

## Next: Part 4 - Hyperparameter Impact

We'll experiment with:
- LoRA rank (r): 4 vs 8 vs 16 vs 32
- Target modules: QV only vs QKVO vs +MLP
- Learning rate: 1e-4 vs 2e-4 vs 5e-4

And use evaluation methods to compare which settings work best!

## Results

![image](Part3_LR_Experiment.PNG)

All three achieved similar final loss (~1.6-1.8) and accuracy (~61%). The warmup schedule showed smoother gradient norms at the start, but didn't dramatically improve final performance.